In [249]:
from copy import deepcopy
from time import time

import torch
import torch.nn.functional as F

from torch import nn
from tensorflow.keras.datasets import fashion_mnist

from torch.utils.data import DataLoader, TensorDataset

import numpy as np
import pandas as pd

In [243]:
class FMnistNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 8, 5)
        self.pool1 = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(8, 16, 5)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(1024, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.softmax(self.fc3(x), dim=1)
        return x

In [244]:
def acc(net_output, labels):
    predicted = net_output.argmax(dim=1)
    correct = (predicted == labels).sum()
    examples = len(labels)
    return (correct / examples).item()

In [245]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

x_train_tensor = torch.tensor(deepcopy(x_train), dtype=torch.float32)
x_train_tensor = x_train_tensor.unsqueeze(1)
x_train_tensor /= 255.

x_test_tensor = torch.tensor(deepcopy(x_test), dtype=torch.float32)
x_test_tensor = x_test_tensor.unsqueeze(1)
x_test_tensor /= 255.

y_train_tensor = torch.tensor(deepcopy(y_train), dtype=torch.long)
y_test_tensor = torch.tensor(deepcopy(y_test), dtype=torch.long)

train_set = DataLoader(TensorDataset(x_train_tensor, y_train_tensor), shuffle=False, batch_size=32)
test_set = DataLoader(TensorDataset(x_test_tensor, y_test_tensor), batch_size=len(x_test_tensor))

In [255]:
net = FMnistNet()
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.01)
epochs = 10

train_loss_progress = []
test_loss_progress = []
train_acc_progress = []
test_acc_progress = []

for epoch in range(epochs):
    epoch_start = time()
    
    accum_loss = []
    accum_acc = []
    
    for data, labels in train_set:
        optimizer.zero_grad()
        net_output = net(data)
        loss = loss_fn(net_output, labels)
        loss.backward()
        optimizer.step()
        
        accum_loss.append(loss.item())
        accum_acc.append(acc(net_output, labels))
        
    train_avg_loss = np.mean(accum_loss)
    train_avg_acc = np.mean(accum_acc)
    
    with torch.no_grad():
        test_data, test_labels = next(iter(test_set))
        test_output = net(test_data)
        test_loss = loss_fn(test_output, test_labels)
        test_acc = acc(test_output, test_labels)
        
    epoch_duration = time() - epoch_start
    
        
    train_loss_progress.append(train_avg_loss)
    test_loss_progress.append(test_loss.item())
    train_acc_progress.append(train_avg_acc)
    test_acc_progress.append(test_acc)
    
    print(f"EPOCH {epoch+1}/{epochs} -- train: (loss: {train_avg_loss:.4f}, acc: {100*train_avg_acc:.2f}%), test: (loss: {test_loss:.4f}, acc: {100*test_acc:.2f}%), time: {epoch_duration:.2f}s")

EPOCH 1/10 -- train: (loss: 2.3020, acc: 14.74%), test: (loss: 2.3013, acc: 22.71%), time: 6.46s
EPOCH 2/10 -- train: (loss: 2.2992, acc: 28.95%), test: (loss: 2.2942, acc: 26.08%), time: 6.56s
EPOCH 3/10 -- train: (loss: 2.0978, acc: 38.20%), test: (loss: 1.9345, acc: 52.27%), time: 6.71s
EPOCH 4/10 -- train: (loss: 1.8673, acc: 59.78%), test: (loss: 1.8038, acc: 67.03%), time: 6.60s
EPOCH 5/10 -- train: (loss: 1.7728, acc: 69.54%), test: (loss: 1.7516, acc: 71.70%), time: 6.50s
EPOCH 6/10 -- train: (loss: 1.7452, acc: 71.96%), test: (loss: 1.7334, acc: 72.89%), time: 6.63s
EPOCH 7/10 -- train: (loss: 1.7250, acc: 73.89%), test: (loss: 1.7196, acc: 74.31%), time: 6.48s
EPOCH 8/10 -- train: (loss: 1.7069, acc: 75.68%), test: (loss: 1.7055, acc: 75.68%), time: 6.48s
EPOCH 9/10 -- train: (loss: 1.6961, acc: 76.78%), test: (loss: 1.7041, acc: 75.88%), time: 6.58s
EPOCH 10/10 -- train: (loss: 1.6878, acc: 77.60%), test: (loss: 1.6908, acc: 77.33%), time: 6.69s


In [263]:
metadata_df = pd.DataFrame((train_loss_progress, test_loss_progress, train_acc_progress, test_acc_progress)).T
metadata_df.columns = ["train_loss", "test_loss", "train_acc", "test_acc"]
metadata_df['epoch'] = np.arange(1, 11)
metadata_df = metadata_df.set_index('epoch')

In [264]:
metadata_df

,train_loss,test_loss,train_acc,test_acc
epoch,,,,
1,2.301998,2.301321,0.147433,0.2271
2,2.299246,2.294165,0.289517,0.2608
3,2.097794,1.934537,0.381983,0.5227
4,1.867267,1.803793,0.597800,0.6703
5,1.772764,1.751600,0.695400,0.7170
6,1.745191,1.733395,0.719633,0.7289
7,1.724972,1.719622,0.738867,0.7431
8,1.706905,1.705550,0.756833,0.7568
9,1.696134,1.704057,0.767800,0.7588
